In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from os.path import join as p_join

import seaborn as sns


##################################
## GLOBAL SETTINGS ###############
##################################
plt.rcParams["figure.figsize"] = (12,8)

In [ ]:
### Install CPU torch
# ! pip3 install torch==1.10.2+cpu torchvision==0.11.3+cpu torchaudio==0.10.2+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html
    
### Inctall GPU torch (CUDA 11.3)
# ! pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [ ]:
###############################
##### IMPORT DL Depensies #####
###############################
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader

from tqdm import tqdm


def seed_all(seed=42):

    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print("[ Using Seed : ", seed, " ]")

####################################
#####   SEED ALL EXPERIMENTS   #####
####################################
seed_all()    


#################################
####### GLOBAL CONFIG ###########
#################################
CONFIG = {'ephs': 5,
          'train_batch_size': 32,
          'test_batch_size': 16,
          'cpu_workers': 2,
          'save_ckpts': True}

In [ ]:
X_path = p_join('..', 'data', 'Input.txt')
Y_path = p_join('..', 'data', 'Topology.txt')

X = pd.read_csv(X_path).values
Y = pd.read_csv(Y_path).values.squeeze()


#################################
#### MAKE TEST TRAIN SPLIT   ####
#################################
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42)


train_dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(Y_train))
test_dataset = TensorDataset(torch.Tensor(X_test), torch.Tensor(Y_test))

train_dataloader = DataLoader(train_dataset,
                              shuffle=True,
                              num_workers=CONFIG['cpu_workers'],
                              batch_size=CONFIG['train_batch_size'],
                              drop_last=True)

test_dataloader = DataLoader(test_dataset,
                             shuffle=False,
                             num_workers=CONFIG['cpu_workers'],
                             batch_size=CONFIG['test_batch_size'],
                             drop_last=False)

In [ ]:
def train_epoch(net, optimizer, dataloader, criterion, device):
    """Perform one traing epoch"""
    running_loss = 0.0
    net.train(True)
    for X, y in tqdm(dataloader):
        X = X.to(device)
        y = y.to(device).long()
        optimizer.zero_grad()
        y_hat = net(X)
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    return running_loss / len(dataloader)

def test_model(net, dataloader, device):
    """Return accuracy"""
    correct, count = 0, 0
    net.eval()
    with torch.no_grad():
        for X, y in tqdm(dataloader):
            X = X.to(device)
            y = y.to(device).long()
            logits = net(X)
            _, y_hat = torch.max(logits, dim=1)
            correct += (y_hat == y).sum().item()
            count += y_hat.shape[0]
    
    return 100 * correct / count

In [ ]:
def run_training(net, optimizer, config, train_loader, test_loader=None):
    ### Get hyperparams from config
    epochs = config.get('ephs', 10)
    device = config.get('device', 'cpu')
    criterion = config.get('criterion', F.cross_entropy)
    ckpt_save_folder = config.get('ckpt_save_folder', './ckpts')
    save_ckpts = config.get('save_ckpts', False)
    
    ### Define local constants
    best_score = -np.inf
    
    if not os.path.exists(ckpt_save_folder):
        os.makedirs(ckpt_save_folder)
    
    ### Start trainig
    for eph in range(1, epochs + 1):
            mean_loss = train_epoch(net, optimizer, train_loader, criterion, device)
            print(f"Epoch: {eph}/{epochs}, \t total train loss: {mean_loss}")
            if test_loader:
                score = test_model(net, test_loader, device)
                print(f"Epoch: {eph}/{epochs}, \t total score test: {score}")
                if score > best_score:
                    best_score = score
                    if save_ckpts:
                        torch.save(net.state_dict(), p_join(ckpt_save_folder, f"model_{eph}_best.ckpt"))
            print()

### Define base model

In [ ]:
class BaseNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.activ = F.leaky_relu
        self.lin1 = nn.Linear(24, 64)
        self.lin2 = nn.Linear(64, 16)
        self.lin3 = nn.Linear(16, 2)

    def forward(self, x):
        x = self.activ(self.lin1(x))
        x = self.activ(self.lin2(x))
        x = torch.tanh(self.lin3(x))
        return x

### Try training:

In [ ]:
CONFIG['lr'] = 0.001
CONFIG['ephs'] = 5
CONFIG['device'] = 'cpu'
CONFIG['criterion'] = F.cross_entropy


net = BaseNet()
optimizer = torch.optim.Adam(net.parameters(), lr=CONFIG.get('lr', 4e-3))
run_training(net, optimizer, CONFIG, train_dataloader, test_dataloader)